# Parse PDFs

This notebook has code to parse a PDF file in the `pdfs` folder into a CSV file in the `data` folder. (I do these one month at a time, spot-checking the data as I go.)

In [28]:
import glob
import os
import csv

import pdfplumber
import pandas as pd

from const import csv_headers, address_fixes

In [2]:
%run download

In [29]:
date = '2024-12'

In [30]:
filepath_in = f'pdfs/{date}.pdf'
filepath_out = f'data/{date}.csv'

year, month = date.split('-')

'''
if os.path.exists(filepath_out):
    raise Exception(f'CSV file exists! {filepath_out}')
'''

# open the PDF with pdfplumber
with pdfplumber.open(filepath_in) as pdf:

    # empty list to hold the extracted data
    table_data = []

    # loop over the PDF pages
    for page in pdf.pages:

        # some reports have 2 tables on the last page,
        # so you need to make sure you end up with the first one
        table = page.extract_tables()[0] 

        # add the data to the tracking list
        if table:
            table_data.extend(table)

# an empty list to hold the parsed data
data_parsed = []

# records are split over two lines, so using sentinel vars
# to guardrail populating a dict as you iterate
current_record = {}

# loop over the rows of raw data
for line in table_data:

    # using some string methods later, so want empty strings
    # instead of None vals for blanks
    line = [x or '' for x in line]

    # skip empty rows
    if not any(line):
        continue

    # see what the first value in the row is
    first_value = line[0].lower()

    # break if you get to a row with any
    # of these terms are present in the first value
    break_terms = [
        'total',
        'year-to-date',
        'year to date',
        'project',
        '\n',
        'permits'
    ]

    if any([x in first_value.lower() for x in break_terms]):
        break

    # parse the line that begins with "address" --
    # second line in the record
    if first_value == 'address':
        _, address,  _, contractor, _, hookup_fee = line

        current_record['site_address'] = address

        current_record['contractor'] = contractor
        current_record['hookup_fee'] = hookup_fee
        current_record['year'] = year
        current_record['month'] = month

        # clean up some garbage chars in the currency columns
        for x in [' ', '$', ',', 'nofee', 'n/a', '-', 'waived']:
            current_record['cost_approximate'] = current_record['cost_approximate'].lower().replace(x, '')
            current_record['permit_fee'] = current_record['permit_fee'].lower().replace(x, '')
            current_record['hookup_fee'] = current_record['hookup_fee'].lower().replace(x, '')

        # we're done! print to let us know what's up
        print(current_record)

        # add the now-full dict to the main list
        data_parsed.append(current_record)

        # ... and reset the tracking dict
        current_record = {}

    # parse the first line of each record
    else:
        permit_id, applicant, const_type, _, cost, permit_fee = line

        # break this out into its own var
        outside_city_limits = ''
    
        if '*' in permit_id:
            outside_city_limits = True
    
        permit_id = permit_id.replace('*', '')

        current_record['permit_number'] = permit_id
        current_record['applicant_name'] = applicant
        current_record['construction_type'] = const_type
        current_record['cost_approximate'] = cost
        current_record['permit_fee'] = permit_fee
        current_record['outside_city_limits'] = outside_city_limits

{'permit_number': 'PMG-24-23', 'applicant_name': 'Rusty karlson', 'construction_type': 'Standalone Permits - Plumbing, Mechanical, Gas', 'cost_approximate': '1.00', 'permit_fee': '56.00', 'outside_city_limits': True, 'site_address': '311 Evans Lane, Unit Lot N, Spearfish SD 57783', 'contractor': 'Owner', 'hookup_fee': '', 'year': '2024', 'month': '12'}
{'permit_number': 'PMG-24-26', 'applicant_name': 'Bryan Robb', 'construction_type': 'Standalone Permits - Plumbing, Mechanical, Gas', 'cost_approximate': '2000.00', 'permit_fee': '51.00', 'outside_city_limits': '', 'site_address': '450 Gooseberry Rd, Spearfish, SD 57783', 'contractor': 'Owner', 'hookup_fee': '', 'year': '2024', 'month': '12'}
{'permit_number': 'PMG-24-24', 'applicant_name': 'Desmond Watson', 'construction_type': 'Standalone Permits - Plumbing, Mechanical, Gas', 'cost_approximate': '340.00', 'permit_fee': '36.00', 'outside_city_limits': '', 'site_address': '1236 SPEARFISH MOUNTAIN LN, SPEARFISH SD', 'contractor': 'Owner',

In [31]:
# write the records to file -- be sure to check the
# record count against the live PDF and check for accuracy
with open(filepath_out, 'w') as outfile:
    writer = csv.DictWriter(outfile, fieldnames=csv_headers)
    writer.writeheader()
    writer.writerows(data_parsed)

print(f'Wrote {len(data_parsed)} records to file: {filepath_out}')

Wrote 15 records to file: data/2024-12.csv


In [32]:
%run combine

Check the address for RND-24-15 (data/2024-12.csv)
    7923 & 7925 DUKE PARKWAY, SPEARFISH, SD 57 57783
Wrote 5,178 permit records to file: spearfish-building-permits.csv
